<a href="https://colab.research.google.com/github/Kavya2099/recommendation-system/blob/main/Project_3_Collaborative_based_movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing data

In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip

--2022-11-02 19:08:44--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip.3’

ml-25m.zip.3        100%[===================>] 249.84M  48.2MB/s    in 5.6s    

2022-11-02 19:08:49 (44.6 MB/s) - ‘ml-25m.zip.3’ saved [261978986/261978986]



In [ ]:
!unzip ml-25m.zip

Archive:  ml-25m.zip
replace ml-25m/tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [ ]:
movie= pd.read_csv('ml-25m/movies.csv')
ratings = pd.read_csv('ml-25m/ratings.csv')

In [ ]:
import re

def new_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

movie["new_title"] = movie["title"].apply(new_title)
movie["new_title"] = movie["new_title"].str.lower()
 

In [ ]:
movie.head()

,movieId,title,genres,new_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,grumpier old men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,waiting to exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,father of the bride part ii 1995


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [ ]:
movie.isnull().sum()

movieId      0
title        0
genres       0
new_title    0
dtype: int64

In [ ]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [ ]:
#merging both dataset
movies= movie.merge(ratings, on='movieId')
movies.head()

,movieId,title,genres,new_title,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,2,3.5,1141415820
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,3,4.0,1439472215
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,4,3.0,1573944252
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,5,4.0,858625949
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,8,4.0,890492517


# Creating tables based on popularity

In [ ]:
#grouping based on total number of ratings given

num_rating= movies.groupby('title').count()['rating'].reset_index()
num_rating.head()

,title,rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",1
1,"""Great Performances"" Cats (1998)",179
2,#1 Cheerleader Camp (2010),9
3,#Captured (2017),2
4,#Female Pleasure (2018),3


In [ ]:
#rename the rating column name to num_of_rating

num_rating.rename(columns = {'rating':'num_of_rating'}, inplace = True)
num_rating.head()

,title,num_of_rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",1
1,"""Great Performances"" Cats (1998)",179
2,#1 Cheerleader Camp (2010),9
3,#Captured (2017),2
4,#Female Pleasure (2018),3


In [ ]:
#grouping based on average number of ratings given

avg_rating= movies.groupby('title').mean()['rating'].reset_index()
avg_rating.head()

,title,rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",3.000000
1,"""Great Performances"" Cats (1998)",2.896648
2,#1 Cheerleader Camp (2010),2.111111
3,#Captured (2017),3.750000
4,#Female Pleasure (2018),3.666667


In [ ]:
#rename the rating column name to num_of_rating

avg_rating.rename(columns = {'rating':'avg_rating'}, inplace = True)
avg_rating

,title,avg_rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",3.000000
1,"""Great Performances"" Cats (1998)",2.896648
2,#1 Cheerleader Camp (2010),2.111111
3,#Captured (2017),3.750000
4,#Female Pleasure (2018),3.666667
...,...,...
58953,…And the Fifth Horseman Is Fear (1965),3.250000
58954,キサラギ (2007),3.250000
58955,チェブラーシカ (2010),3.166667
58956,貞子3D (2012),2.375000


In [ ]:
#merge two tables

popularity = num_rating.merge(avg_rating,on='title')
popularity.head()


,title,num_of_rating,avg_rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",1,3.000000
1,"""Great Performances"" Cats (1998)",179,2.896648
2,#1 Cheerleader Camp (2010),9,2.111111
3,#Captured (2017),2,3.750000
4,#Female Pleasure (2018),3,3.666667


In [34]:
# only taking the titles where users have given number of ratings > 1000 and sorting the titles in descending based on average rating

popularity = popularity[popularity.num_of_rating > 1000].sort_values('avg_rating',ascending=False)
#this shows top 50 popular movies in the dataset
popularity.head(50)

,title,num_of_rating,avg_rating
36804,Planet Earth II (2016),1124,4.483096
36803,Planet Earth (2006),1747,4.464797
41782,"Shawshank Redemption, The (1994)",81482,4.413576
4934,Band of Brothers (2001),1356,4.398599
19301,"Godfather, The (1972)",52498,4.324336
55289,"Usual Suspects, The (1995)",55366,4.284353
19302,"Godfather: Part II, The (1974)",34188,4.261759
41448,Seven Samurai (Shichinin no samurai) (1954),13367,4.254769
40816,Schindler's List (1993),60411,4.247579
182,12 Angry Men (1957),16569,4.243014


# Collaborative based filtering

In [ ]:
#group by userids who has given ratings more than 1500 movies
x= movies.groupby('userId').count()['rating']>1500
x.head()

userId
1    False
2    False
3    False
4    False
5    False
Name: rating, dtype: bool

In [ ]:
#getting their index of users who has given ratings for more than 1000 movies; 
sim_users= x[x].index 
sim_users

Int64Index([   548,    626,    847,    997,   1401,   1652,   1748,   1920,
              1977,   2165,
            ...
            160922, 160951, 161184, 161383, 161544, 161586, 161928, 162047,
            162271, 162516],
           dtype='int64', name='userId', length=910)

In [ ]:
#all movies that sim_users has also viewed
sim_movies= movies[movies['userId'].isin(sim_users)]
sim_movies

,movieId,title,genres,new_title,userId,rating,timestamp
172,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,548,4.5,1431644949
199,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,626,4.5,1136304145
283,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,847,4.0,1048092664
339,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,997,4.5,1529250285
477,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,1401,4.5,1544250123
...,...,...,...,...,...,...,...
25000061,209067,Sousse: Marché aux charbons (avec chameaux) (1...,(no genres listed),sousse march aux charbons avec chameaux 1896,154484,2.5,1574026518
25000068,209089,An Impossible Balancing Feat (1902),(no genres listed),an impossible balancing feat 1902,154484,3.0,1574075308
25000070,209103,Tsar Ivan the Terrible (1991),(no genres listed),tsar ivan the terrible 1991,13737,4.0,1574112239
25000079,209135,Jane B. by Agnès V. (1988),Documentary|Fantasy,jane b by agns v 1988,154484,3.5,1574187267


In [ ]:
#groupby title for movies the number of ratings given to the movie is greater than 500
y= sim_movies.groupby('title').count()['rating']>500
#getting their index
famous_movies= y[y].index 
famous_movies

Index(['(500) Days of Summer (2009)', '10 Things I Hate About You (1999)',
       '101 Dalmatians (One Hundred and One Dalmatians) (1961)',
       '12 Angry Men (1957)', '2001: A Space Odyssey (1968)',
       '21 Grams (2003)', '28 Days Later (2002)', '300 (2007)',
       '3:10 to Yuma (2007)', '40-Year-Old Virgin, The (2005)',
       ...
       'X-Men (2000)', 'X-Men: First Class (2011)',
       'X-Men: The Last Stand (2006)', 'X2: X-Men United (2003)',
       'You've Got Mail (1998)', 'Young Frankenstein (1974)', 'Zodiac (2007)',
       'Zombieland (2009)', 'Zoolander (2001)', 'xXx (2002)'],
      dtype='object', name='title', length=751)

In [ ]:
#getting the details of the movies where the number of ratings given to the movie is greater than 500 
final_ratings= sim_movies[movies['title'].isin(famous_movies)]
final_ratings

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,movieId,title,genres,new_title,userId,rating,timestamp
172,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,548,4.5,1431644949
199,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,626,4.5,1136304145
283,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,847,4.0,1048092664
339,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,997,4.5,1529250285
477,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,1401,4.5,1544250123
...,...,...,...,...,...,...,...
24454291,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller,confessions of a dangerous mind 2002,150655,4.0,1571929399
24454293,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller,confessions of a dangerous mind 2002,153086,3.5,1494520954
24454298,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller,confessions of a dangerous mind 2002,160747,2.0,1476826244
24454300,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller,confessions of a dangerous mind 2002,161544,3.5,1499019766


In [ ]:
final_ratings.drop_duplicates()

,movieId,title,genres,new_title,userId,rating,timestamp
172,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,548,4.5,1431644949
199,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,626,4.5,1136304145
283,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,847,4.0,1048092664
339,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,997,4.5,1529250285
477,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story 1995,1401,4.5,1544250123
...,...,...,...,...,...,...,...
24454291,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller,confessions of a dangerous mind 2002,150655,4.0,1571929399
24454293,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller,confessions of a dangerous mind 2002,153086,3.5,1494520954
24454298,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller,confessions of a dangerous mind 2002,160747,2.0,1476826244
24454300,144606,Confessions of a Dangerous Mind (2002),Comedy|Crime|Drama|Romance|Thriller,confessions of a dangerous mind 2002,161544,3.5,1499019766


We can see that there is no duplicates

> We are using pivot table instead of TfidfVectorizer. TfidfVectorizer converts a collection of raw documents to a matrix of TF-IDF features. 

> Here we dont have any text except title to give as input to the vectorizer. 

> If we use this, we'll end up getting recooemdations based on title only not based on other users

> Because of that reason we are using pivot table.

In [54]:
"""
Creating a pivot table

A PivotTable is an interactive way to quickly summarize large amounts of data. You can use a PivotTable to analyze numerical data in detail """

pt= final_ratings.pivot_table(index='new_title',columns='userId',values='rating')
#pt1 is for getting/printing the actual/original title of the movie of the same index as of pt
pt1= final_ratings.pivot_table(index=['new_title','title'],columns='userId',values='rating')

pt

userId,548,626,847,997,1401,1652,1748,1920,1977,2165,...,160922,160951,161184,161383,161544,161586,161928,162047,162271,162516
new_title,,,,,,,,,,,,,,,,,,,,,
10 things i hate about you 1999,4.5,4.0,3.5,3.0,1.0,NaN,1.5,NaN,NaN,4.0,...,4.0,3.0,2.5,NaN,NaN,NaN,NaN,3.0,NaN,3.5
101 dalmatians one hundred and one dalmatians 1961,4.0,4.0,3.5,3.0,NaN,5.0,3.0,3.0,4.0,NaN,...,4.0,NaN,4.0,3.5,NaN,2.5,3.0,NaN,NaN,4.0
12 angry men 1957,NaN,4.0,4.0,4.0,5.0,5.0,NaN,NaN,4.0,5.0,...,4.0,NaN,5.0,NaN,NaN,3.5,NaN,5.0,3.5,4.5
2001 a space odyssey 1968,NaN,4.0,4.5,5.0,3.0,3.0,4.0,4.0,4.5,0.5,...,3.0,4.5,5.0,3.0,4.0,5.0,NaN,4.0,4.5,4.5
21 grams 2003,4.0,NaN,2.5,4.0,NaN,NaN,3.5,4.0,3.0,5.0,...,NaN,4.0,4.5,NaN,3.0,NaN,NaN,4.0,2.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
young frankenstein 1974,NaN,4.0,3.0,NaN,2.0,NaN,1.5,4.0,NaN,NaN,...,4.5,4.5,4.5,3.0,NaN,4.5,NaN,2.0,3.5,4.0
youve got mail 1998,3.5,4.0,3.5,3.0,NaN,4.0,4.0,3.0,2.0,NaN,...,3.5,3.0,3.0,NaN,1.5,NaN,3.0,2.5,1.5,2.5
zodiac 2007,4.5,NaN,4.0,4.0,4.0,5.0,3.5,3.0,NaN,1.5,...,3.0,4.0,3.0,NaN,2.5,2.0,NaN,5.0,4.0,4.0


In [55]:
pt.fillna(0,inplace=True)
pt1.fillna(0,inplace=True)
pt

userId,548,626,847,997,1401,1652,1748,1920,1977,2165,...,160922,160951,161184,161383,161544,161586,161928,162047,162271,162516
new_title,,,,,,,,,,,,,,,,,,,,,
10 things i hate about you 1999,4.5,4.0,3.5,3.0,1.0,0.0,1.5,0.0,0.0,4.0,...,4.0,3.0,2.5,0.0,0.0,0.0,0.0,3.0,0.0,3.5
101 dalmatians one hundred and one dalmatians 1961,4.0,4.0,3.5,3.0,0.0,5.0,3.0,3.0,4.0,0.0,...,4.0,0.0,4.0,3.5,0.0,2.5,3.0,0.0,0.0,4.0
12 angry men 1957,0.0,4.0,4.0,4.0,5.0,5.0,0.0,0.0,4.0,5.0,...,4.0,0.0,5.0,0.0,0.0,3.5,0.0,5.0,3.5,4.5
2001 a space odyssey 1968,0.0,4.0,4.5,5.0,3.0,3.0,4.0,4.0,4.5,0.5,...,3.0,4.5,5.0,3.0,4.0,5.0,0.0,4.0,4.5,4.5
21 grams 2003,4.0,0.0,2.5,4.0,0.0,0.0,3.5,4.0,3.0,5.0,...,0.0,4.0,4.5,0.0,3.0,0.0,0.0,4.0,2.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
young frankenstein 1974,0.0,4.0,3.0,0.0,2.0,0.0,1.5,4.0,0.0,0.0,...,4.5,4.5,4.5,3.0,0.0,4.5,0.0,2.0,3.5,4.0
youve got mail 1998,3.5,4.0,3.5,3.0,0.0,4.0,4.0,3.0,2.0,0.0,...,3.5,3.0,3.0,0.0,1.5,0.0,3.0,2.5,1.5,2.5
zodiac 2007,4.5,0.0,4.0,4.0,4.0,5.0,3.5,3.0,0.0,1.5,...,3.0,4.0,3.0,0.0,2.5,2.0,0.0,5.0,4.0,4.0


In [ ]:
#cosine simularity
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(pt)
similarity_scores.shape

(751, 751)

In [ ]:
similarity_scores

array([[1.        , 0.59671396, 0.58509239, ..., 0.62510637, 0.61193687,
        0.67061426],
       [0.59671396, 1.        , 0.63601144, ..., 0.52034784, 0.52022317,
        0.60651434],
       [0.58509239, 0.63601144, 1.        , ..., 0.6929618 , 0.61668775,
        0.64435058],
       ...,
       [0.62510637, 0.52034784, 0.6929618 , ..., 1.        , 0.74656204,
        0.65302658],
       [0.61193687, 0.52022317, 0.61668775, ..., 0.74656204, 1.        ,
        0.68600489],
       [0.67061426, 0.60651434, 0.64435058, ..., 0.65302658, 0.68600489,
        1.        ]])

In [56]:
#creating function

def recommend_movie1(title):
  title= title.lower()
  title= new_title(title)
  #print(title)
  #getting index; last two zeros denotes getting the index value from set
  index= np.where(pt.index==title)[0][0]
  #print(index)
  sim_scores= enumerate(similarity_scores[index])
  sim_scores= sorted(sim_scores, key= lambda x: x[1], reverse=True)
  sim_scores= sim_scores[1:11]
  
  #getting the index of the sim_scores
  sim_index= [i[0] for i in sim_scores]
  
  for i in sim_index:
    print(pt1.index[i][1])


In [57]:
recommend_movie1('Raging Bull (1980)')

Taxi Driver (1976)
Apocalypse Now (1979)
Chinatown (1974)
Godfather, The (1972)
Goodfellas (1990)
Godfather: Part II, The (1974)
Citizen Kane (1941)
One Flew Over the Cuckoo's Nest (1975)
Deer Hunter, The (1978)
Fargo (1996)




> This'll work only when we provide movies along with year. If we provide without year we'll get error. We'll add exception to it

> The major reason for providing with year is that some movies have same movie name but released in different year. Ex: Planet of the Apes

> In those cases if we try to use w/o year, we might end up different/incorrect recommendations. we'll modify the title without year and since we are using the same title as index, and dropping duplicates in pivot table, we might end up with losing movies information

> Due to that reason we are trying recoomendation with year


In [63]:
#tring with exception
from logging import exception
#creating function

def recommend_movie(title):
  try:
    title= title.lower()
    title= new_title(title)
    #print(title)
    #getting index; last two zeros denotes getting the index value from set
    index= np.where(pt.index==title)[0][0]
    #print(index)
    sim_scores= enumerate(similarity_scores[index])
    sim_scores= sorted(sim_scores, key= lambda x: x[1], reverse=True)
    sim_scores= sim_scores[1:11]
    
    #getting the index of the sim_scores
    sim_index= [i[0] for i in sim_scores]
    
    for i in sim_index:
      print(pt1.index[i][1])
  except:
    print('Given input title is either misspelt or not found in our dataset or given without year for movie recommendation.\nKindly provide different title or with year. Thank you!')

In [64]:
recommend_movie('Raging Bull ')

Given input title is either misspelt or not found in our dataset or given without year for movie recommendation.
Kindly provide different title or with year. Thank you!


In [65]:
recommend_movie('Raging Buil (1980)')

Given input title is either misspelt or not found in our dataset or given without year for movie recommendation.
Kindly provide different title or with year. Thank you!


In [66]:
recommend_movie('Raging Bull (1980)')

Taxi Driver (1976)
Apocalypse Now (1979)
Chinatown (1974)
Godfather, The (1972)
Goodfellas (1990)
Godfather: Part II, The (1974)
Citizen Kane (1941)
One Flew Over the Cuckoo's Nest (1975)
Deer Hunter, The (1978)
Fargo (1996)


In [67]:
recommend_movie('12 Angry Men (1957)')

One Flew Over the Cuckoo's Nest (1975)
Rear Window (1954)
Godfather, The (1972)
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
Casablanca (1942)
Fargo (1996)
Goodfellas (1990)
Taxi Driver (1976)
Pulp Fiction (1994)
2001: A Space Odyssey (1968)


In [68]:
recommend_movie('101 dalmatians one hundred and one dalmatians 1961')

Lady and the Tramp (1955)
Bambi (1942)
Snow White and the Seven Dwarfs (1937)
Cinderella (1950)
Jungle Book, The (1967)
Pinocchio (1940)
Dumbo (1941)
Beauty and the Beast (1991)
Mary Poppins (1964)
Little Mermaid, The (1989)
